In [80]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta

# Options Data from Yahoo

In [38]:
#Webscrapping code thanks to Tony Lian
# https://medium.com/@txlian13/webscrapping-options-data-with-python-and-yfinance-e4deb0124613
def options_chain(symbol):

    tk = yf.Ticker(symbol)
    # Expiration dates
    exps = tk.options

    # Get options for each expiration
    options = pd.DataFrame()
    for e in exps:
        opt = tk.option_chain(e)
        opt = pd.DataFrame().append(opt.calls).append(opt.puts)
        opt['expirationDate'] = e
        options = options.append(opt, ignore_index=True)

    # Bizarre error in yfinance that gives the wrong expiration date
    # Add 1 day to get the correct expiration date
    options['expirationDate'] = pd.to_datetime(options['expirationDate']) + timedelta(days = 1)
    options['dte'] = (options['expirationDate'] - datetime.now()).dt.days / 365
    
    # Boolean column if the option is a CALL
    options['CALL'] = options['contractSymbol'].str[4:].apply(
        lambda x: "C" in x)
    
    options[['bid', 'ask', 'strike']] = options[['bid', 'ask', 'strike']].apply(pd.to_numeric)
    options['mark'] = (options['bid'] + options['ask']) / 2 # Calculate the midpoint of the bid-ask
    
    # Drop unnecessary and meaningless columns
    options = options.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate', 'lastPrice'])

    return options

In [46]:
#Run this at the end of every weekday
options = options_chain('^SPX')

In [52]:
#Save to csv
options.to_csv('SPX_options_chain_' + str(date.today()))

In [53]:
def extract_observed_vol_surface(df):
    

,contractSymbol,strike,bid,ask,volume,openInterest,impliedVolatility,inTheMoney,expirationDate,dte,CALL,mark
0,SPXW220408C03250000,3250.0,1225.5,1249.3,18.0,15.0,3.504274,1.0,2022-04-09,0.00000,True,1237.40
1,SPXW220408C03350000,3350.0,1125.5,1149.3,2.0,2.0,3.225222,1.0,2022-04-09,0.00000,True,1137.40
2,SPXW220408C03450000,3450.0,1025.5,1049.3,1.0,16.0,2.951907,1.0,2022-04-09,0.00000,True,1037.40
3,SPXW220408C03500000,3500.0,978.6,999.5,15.0,15.0,1.895020,1.0,2022-04-09,0.00000,True,989.05
4,SPXW220408C03550000,3550.0,926.1,949.5,1.0,1.0,2.693302,1.0,2022-04-09,0.00000,True,937.80
...,...,...,...,...,...,...,...,...,...,...,...,...
11945,SPX261218P08600000,8600.0,0.0,0.0,5.0,0.0,0.000010,1.0,2026-12-19,4.69863,False,0.00
11946,SPX261218P08800000,8800.0,3303.4,3603.4,1.0,3.0,0.000010,1.0,2026-12-19,4.69863,False,3453.40
11947,SPX261218P09000000,9000.0,3473.0,3773.0,1.0,2.0,0.000010,1.0,2026-12-19,4.69863,False,3623.00
11948,SPX261218P09200000,9200.0,0.0,0.0,2.0,11.0,0.000010,1.0,2026-12-19,4.69863,False,0.00


In [154]:
calls = options[options['CALL'] == True]


array([4488.28, 4488.28, 4488.28, ..., 4488.28, 4488.28, 4488.28])

In [160]:
calls['close'] = (4488.28)*np.ones(len(calls.index))

/Users/liamsantacruz/miniconda3/envs/python_env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [162]:
calls =calls[calls['expirationDate'] == pd.Timestamp('2022-04-09 00:00:00', tz=None)]

In [163]:
df1 = calls[['strike','expirationDate','impliedVolatility', 'close']]

In [164]:
timeToExp = []
for ts in df1['expirationDate']:
    tte = int(str(ts - pd.Timestamp('2021-04-08 00:00:00', tz=None)).split()[0])/256
    timeToExp.append(tte)

In [165]:
df1['timeToExp'] = timeToExp

/Users/liamsantacruz/miniconda3/envs/python_env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [168]:
df1

,strike,expirationDate,impliedVolatility,close,timeToExp
0,3250.0,2022-04-09,3.504274,4488.28,1.429688
1,3350.0,2022-04-09,3.225222,4488.28,1.429688
2,3450.0,2022-04-09,2.951907,4488.28,1.429688
3,3500.0,2022-04-09,1.895020,4488.28,1.429688
4,3550.0,2022-04-09,2.693302,4488.28,1.429688
...,...,...,...,...,...
197,5050.0,2022-04-09,0.691409,4488.28,1.429688
198,5100.0,2022-04-09,0.742190,4488.28,1.429688
199,5200.0,2022-04-09,0.847658,4488.28,1.429688
200,5400.0,2022-04-09,1.046880,4488.28,1.429688


1.42578125